In [43]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [44]:
df = pd.read_csv("./bankDataset/bank-full.csv",sep = ";")

In [45]:
#Encode data using LabelEncoder
for column in df.select_dtypes(include=['object']).columns:
    df[column] = LabelEncoder().fit_transform(df[column])


In [34]:
X = df.drop("y",axis = 1)
y = df["y"]

In [36]:
X_train,X_test,y_train_true,y_test_true = train_test_split(X,y,test_size=0.2, random_state=42)

In [38]:
#Division of into labeled and unlabeled
X_label,y_label = X_train[:400], y_train_true[:400]
X_unlabel = X_train[400:]

## SEMI-SUPERVISED ENSEMBLE

In [46]:
tree_clf = DecisionTreeClassifier(random_state = 42)
svm_clf = SVC(probability=True, random_state=42)

tree_clf.fit(X_label,y_label)
svm_clf.fit(X_label,y_label)

# Pseudo-labeling on unlabeled data
pseudo_labels = np.array([tree_clf.predict(X_unlabel), svm_clf.predict(X_unlabel)])
confidence = np.max([tree_clf.predict_proba(X_unlabel), svm_clf.predict_proba(X_unlabel)], axis=0)
threshold = 0.75  # You can adjust this threshold
high_confidence = np.max(confidence, axis=1) > threshold
X_pseudo_labeled = X_unlabel[high_confidence]
y_pseudo_labeled = np.round(np.mean(pseudo_labels[:, high_confidence], axis=0)).astype(int)

# Combine into an ensemble and retrain
ensemble_clf = VotingClassifier(estimators=[('dt', tree_clf), ('svm', svm_clf)], voting='soft')

# Combine labeled and pseudo-labeled data
X_combined = np.vstack((X_label, X_pseudo_labeled))
y_combined = np.hstack((y_label, y_pseudo_labeled))

ensemble_clf.fit(X_combined, y_combined)

# Evaluate the model
y_pred = ensemble_clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test_true, y_pred))

/usr/lib/python3/dist-packages/sklearn/base.py:413: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/usr/lib/python3/dist-packages/sklearn/base.py:413: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


Accuracy: 0.8791330310737587


## UNSUPERVISED PRE-TRAINED

In [47]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'